In [1]:
import numpy as np
import pandas as pd
import math
from statsmodels import regression

In [2]:
data = pd.read_csv("../data/Case3HistoricalPrices.csv")
data = data.drop(columns=['Unnamed: 0'])
data.head(-5)

,S1,S2,S3,S4,S5,S6,S7,S8,B1,B2,...,B7,B8,C1,C2,C3,C4,C5,C6,C7,C8
0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,98.355850,98.307641,95.630621,98.296988,98.474008,95.663069,98.342292,98.416837,100.086605,99.685232,...,99.674428,100.131478,95.854652,95.571202,94.085760,93.956485,94.140624,94.058865,95.786959,94.331430
2,96.766305,97.785655,95.211317,97.876191,96.910785,95.299426,97.862363,96.850881,100.595828,99.614514,...,99.463006,100.594669,87.500105,86.517650,88.861746,89.205952,89.255488,89.203200,86.873750,89.644765
3,99.498201,97.762932,95.831525,97.979482,99.992847,95.890271,97.890721,99.523488,100.687198,99.425275,...,99.220046,100.639175,84.500885,83.306741,86.183137,86.617107,86.643905,86.512592,83.788306,87.068357
4,96.259221,92.942010,94.272932,92.958011,96.469517,94.318577,93.007052,96.278064,101.939638,100.099827,...,99.843072,101.823484,79.974708,78.434282,80.585967,81.404902,81.484117,81.131945,79.157682,81.778226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,329.571663,332.038448,214.785331,268.624233,343.032460,110.231042,280.749048,451.478031,148.209537,145.528211,...,101.304242,164.288945,107.134821,86.988745,112.066514,59.802608,123.115258,165.989411,171.152575,93.702408
2511,333.241086,330.111672,212.581270,263.525020,347.112306,108.786471,279.410524,455.416775,148.666454,146.039146,...,101.227410,164.728097,107.284645,86.392955,112.446308,58.987521,122.068295,167.115678,170.716373,92.786356
2512,330.911166,327.267712,208.599873,258.850787,344.382675,109.801893,276.229074,453.069496,148.596464,145.782165,...,101.046012,164.217254,108.147948,86.707331,113.868806,59.554939,123.093553,168.137813,170.582716,92.188027
2513,329.808903,328.875381,207.622462,261.904975,343.104107,110.374945,276.415575,451.386406,148.861887,145.597568,...,100.763608,164.831889,107.793831,86.789811,113.321746,59.637080,123.042712,167.505370,170.763926,92.717958


In [3]:
train_split = 252*5 #5 Years
train = data.iloc[:train_split].reset_index(drop=True)
test = data.iloc[train_split:].reset_index(drop=True)

# Method 1:

Run OLS to get $b_0$ and $b_1$, where $b_0$ is the intercept and $b_1$ is the slope. Buy all stocks that have a positive $b_1$, and don't touch those assets.

In [4]:
# Initialize global variables
b1 = []
b0 = []
x = np.array([i for i in range(len(data))])
x = np.column_stack((x, [1]*len(x)))
for i in range(len(data.columns)):
    name = data.columns[i]
    prices = data[name].to_numpy()
    model = regression.linear_model.OLS(prices, x).fit()
    b1.append(model.params[0])
    b0.append(model.params[1])

def allocate1(asset_prices):
    portfolio = np.array([1 if b1[i] > 0 else 0 for i in range(len(b1))])
    if sum(portfolio): return portfolio / sum(portfolio) # Return normalized weights
    else: return portfolio

### Results

In [5]:
# Annualized daily sharpe ratio
weights = np.zeros(len(data.columns))
prev_row = np.zeros(len(data.columns))
returns = []

for index, row in test.iterrows():
    if index == 0: # No returns on first day
        prev_row = row.to_numpy()
        new_weights = allocate1(prev_row)
    else: # Add daily return
        asset_prices = row.to_numpy()
        yesterday = prev_row * weights
        today = asset_prices * new_weights
        transaction_fee = 30*sum(abs(new_weights - weights))
        daily_return = sum(today) - sum(yesterday)
        returns.append(daily_return)
        # Update prev_row and weights
        prev_row = asset_prices
        weights = new_weights
        new_weights = allocate1(asset_prices)

In [6]:
total = 0
for i in range(1,6):
    year_ret = np.array(returns[(i-1)*252: i*252])
    if year_ret.std():
        ratio = math.sqrt(252)*year_ret.mean()/year_ret.std()
    else: ratio = 0
    total += ratio
    print("Sharpe ratio for year ", i, ": ", ratio)
print("Total points: ", total)

Sharpe ratio for year  1 :  0.9202531819092468
Sharpe ratio for year  2 :  1.5551529344237345
Sharpe ratio for year  3 :  0.15048333624809024
Sharpe ratio for year  4 :  1.0404178326300075
Sharpe ratio for year  5 :  -0.5528437461885013
Total points:  3.113463539022577


# Method 2:

Run OLS to get $b_0$ and $b_1$. Sell all stocks that are a standard deviation above the OLS price, and buy all stocks that are a standard deviation below the OLS price.

In [19]:
# Initialize global variables
b1 = []
b0 = []
std_err = []
x = np.array([i for i in range(len(data))])
x = np.column_stack((x, [1]*len(x)))
for i in range(len(data.columns)):
    name = data.columns[i]
    prices = data[name].to_numpy()
    model = regression.linear_model.OLS(prices, x).fit()
    b1.append(model.params[0])
    b0.append(model.params[1])
    line = x.T[0]*b1[i] + b0[i]
    adj_prices = prices - line
    std_err.append(np.std(adj_prices))

portfolio = np.zeros(len(data.columns))
t = len(train)
def allocate2(asset_prices):
    global t
    for i in range(len(asset_prices)):
        ols_price = t*b1[i] + b0[i]
        if asset_prices[i] > ols_price + 2*std_err[i]:
            portfolio[i] += 1 # Buy
        elif asset_prices[i] < ols_price - 2*std_err[i]:
            portfolio[i] = max(portfolio[i] - 1, 0) #Sell
    t += 1
    if sum(portfolio): return portfolio / sum(portfolio) # Return normalized weights
    else: return portfolio

### Results

If you take out transation fee, the total points becomes positive

In [20]:
# Annualized daily sharpe ratio
weights = np.zeros(len(data.columns))
prev_row = np.zeros(len(data.columns))
returns = []

for index, row in test.iterrows():
    if index == 0: # No returns on first day
        prev_row = row.to_numpy()
        new_weights = allocate2(prev_row)
    else: # Add daily return
        asset_prices = row.to_numpy()
        yesterday = prev_row * weights
        today = asset_prices * new_weights
        transaction_fee = 30*sum(abs(new_weights - weights))
        daily_return = sum(today) - sum(yesterday) - transaction_fee
        returns.append(daily_return)
        # Update prev_row and weights
        prev_row = asset_prices
        weights = new_weights
        new_weights = allocate2(asset_prices)

In [21]:
total = 0
for i in range(1,6):
    year_ret = np.array(returns[(i-1)*252: i*252])
    if year_ret.std():
        ratio = math.sqrt(252)*year_ret.mean()/year_ret.std()
    else: ratio = 0
    total += ratio
    print("Sharpe ratio for year ", i, ": ", ratio)
print("Total points: ", total)

Sharpe ratio for year  1 :  0.9349698359350319
Sharpe ratio for year  2 :  -0.96911095773847
Sharpe ratio for year  3 :  2.2480964907916916
Sharpe ratio for year  4 :  1.4168211716593984
Sharpe ratio for year  5 :  -2.3610148569758973
Total points:  1.2697616836717547
